# Intro to Classification 03


## Churn Prediction!

### Your name:

#### 실습목표<br>
장바구니 데이터 부터, 이탈 예측 까지.
---------------
Rayleigh Kim

In [0]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

## 주의

Churn_Prediction_I번 파일의 계획서와 같이 보는 것이 좋다!

우리는 Feature Engineering, Model Selection이라는 것을 해볼 예정이다!


동시에 너무너무 힘든 사람들은, **(new주석, 공개)RetailChurnTemplate_FeatureEngg.xlsx** 파일안의 Feature들을 만드는 연습이라도 해보면 좋다.

위의 엑셀 파일은,  Labelling 작업과 Feature Engineering이 완료되어 바로 ML을 해볼 수 있는 형태의 데이터다.  일종의 '가답안'이다.


### 지금 이 곳 에선.

일단 최근 21일 사이에 구매가 없는 사람을 이탈이라 정의하겠다!

## Data Loading

    

In [0]:
'''
고객 정보 데이터
'''
user_info = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_UserInfoData.csv')
user_info.head()

In [0]:
## 각 컬럼별로, 유니크한 값이 몇 개나 있을까?
user_col_list = list(user_info)

for col in user_col_list :
    print('컬럼명 : {},  안에 유니크한 값의 갯수 : {}'.format(col, len(user_info[col].unique())))

In [0]:
## Gender와 UserType 하등 쓸모 없다. 버리자.
user_info = user_info[['UserId', 'Age', 'Address']]
user_info.head()

**미세한 팁!**

왜, 아래와 같이 불러오는지 생각해보자!

In [0]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/transaction_sample.csv',
                        nrows=3)
raw_data.head()

In [0]:
raw_data.dtypes

In [0]:
'''
Transaction 데이터
'''

raw_data = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/transaction_sample.csv')
raw_data.head()

In [0]:
raw_data['ItemId'][0]

In [0]:
## 각 컬럼별로, 유니크한 값이 몇 개나 있을까?
tr_col_list = list(raw_data)

for col in tr_col_list :
    print('컬럼명 : {},  안에 유니크한 값의 갯수 : {}'.format(col, len(raw_data[col].unique())))

In [0]:
## Location, ProductCategory 하등 쓸모 없다. 버리자.
raw_data = raw_data[tr_col_list[:-2]] # 우연히 마지막 두 개가 쓸모 없어서 이렇게 처리했다.
raw_data.head()

In [0]:
raw_data.tail()

In [0]:
raw_data.shape

In [0]:
raw_data.dtypes

In [0]:
raw_data['Timestamp'] = pd.to_datetime(raw_data['Timestamp'])

In [0]:
raw_data['Timestamp'].min()

In [0]:
raw_data['Timestamp'].max()

In [0]:
## ItemId 와, Quantity와 Value의 관계를 알기 위해서 관찰해봐야 한다.
raw_data.sort_values(by = 'ItemId')

## Data Preprocessing I



최근 21일 사이에 오지 않으면 이탈이다.

**첫 번째 제거해야할 지점.**<br>
==> 최근 21일 사이에 첫 구매인 고객은, 이탈 정의상 고려할 수 없다.

==> 일단 다 지워버려야 한다!

In [0]:
## 컬럼이름들
list(raw_data)

In [0]:
## 가장 최근일자 저장
latest_day = raw_data['Timestamp'].max()

## 최근 21일이 이탈의 기준!
churn_period = 21

## 이탈 기준일을 제작하자
churn_thres_day = raw_data['Timestamp'].max() - pd.to_timedelta(churn_period, unit='d')
print('이탈 기준일 : {}'.format(churn_thres_day))

## 유저별로 최초의 구매일만을 남겨보자!
users_first_days = raw_data.groupby(by=['UserId'], as_index = False)['Timestamp'].min()

if 'Timestamp' in users_first_days.columns :
    users_first_days.rename(columns={'Timestamp':'first_day'}, inplace = True)

print(users_first_days.head())

In [0]:
## 최초의 구매일이, 이탈 기준일 이전인 사람들의 UserId를 리스트에 담아두자
not_new_users = users_first_days['UserId'].loc[users_first_days['first_day'] <= churn_thres_day  ].tolist()

## 원래 row수 카운트
print('원래 row수는 {}'.format(raw_data.shape[0]))

## 위의, 이탈 기준일 이전부터 구매해오던 사람들의 데이터만 남기자.
processed_data_i = raw_data[raw_data['UserId'].isin(not_new_users)]


## 제거 후 row수 카운트
print('신규 회원 제거 후 row수는 {}'.format(processed_data_i.shape[0]))

print('신규 회원 제거 전, 고객 수는 {}'.format(len(raw_data['UserId'].unique())))
print('신규 회원 제거 후, 고객 수는 {}'.format(len(processed_data_i['UserId'].unique())))

## Data Preprocessing II

### 이 문제를 풀기 위한, 데이터 분리.

1. Labelling, 즉 '이탈자 / 비이탈자' 여부는  **최근 21일 사이** 구매가 있었는지 없었는지로 판단한다.

2. Label, Target을 설명하기 위한 Input으로는 **최근 21일을 제외한** 데이터를 요약 하여 사용해야 한다.

**그래야 예측 아니겠는가!**

### Labelling!

그야말로 튜토리얼과 현실을 가르는 지점 중 하나다.<br>
연습용 데이터는 항상 'Label, Target'이 있었다. <br>
실전에선 그것 부터 만들고 시작하는 경우가 허다함!

Labelling 규칙 (인코딩 규칙)

Churner : 1
Non-churner : 0

In [0]:
# UserId 별로, 가장 최근 구매일자만 담긴 데이터 프레임을 만들자.
temp_i = processed_data_i.groupby(by=['UserId'], as_index = False)['Timestamp'].max()
if 'Timestamp' in temp_i.columns :
    temp_i.rename(columns={'Timestamp':'last_day'}, inplace = True)
temp_i.head()

In [0]:
# 일단 전부 Non-churner : 0 이라고 Label을 달아둔다.
temp_i['label'] = 0
temp_i.head(2)

In [0]:
# last_day가, 이탈 기준일 이전인 사람 (최근 21일간 거래내역이 없는 사람)
# 그런 사람들만 Churner 다!
temp_i['label'].loc[temp_i['last_day'] <= churn_thres_day  ] = 1

# churner를 만들어서, UserId와 label만 담긴 데이터 프레임을 따로 떼어 둔다.
labels = temp_i[['UserId', 'label']]

# temp_i를 한 번 보자.
temp_i.head(8)

### Data set for Feature Engineering!

Remind! : Input은, 최근 21일 사이의 데이터를 제하고 만드는 것이다!

In [0]:
## 최근 21일 사이 데이터 제거
before_21 = processed_data_i.loc[processed_data_i['Timestamp'] <= churn_thres_day  ]

## 제거 전 row 수
print('최근 21일 데이터 제거 전 row수 : {}'.format(processed_data_i.shape[0]))
print('최근 21일 데이터 제거 후 row수 : {}'.format(before_21.shape[0]))

## Data Preprocessing III : Feature Engineering

### 한 번 쯤 정리하고 가자, 지금까지 손에 들어온 데이터가 무엇 무엇이 있는가?

In [0]:
user_info.head(2)

In [0]:
labels.head(2)

In [0]:
before_21.head(2)

In [0]:
## Label 데이터 안에 있는 UserId 수
len(labels['UserId'].unique())

In [0]:
## 최근 21일 뗀 거래 내역에 있는 UserId 수
len(before_21['UserId'].unique())

### Feature Engineering

주의 :
1. 가능하면 무엇을 만들어 낼 것인지 사전에 계획을 세워둘 것.
2. 만들어낸 Feature가 왜 중요할지, 거친 추측이라도 해볼 것.
3. 풍부히 만들 것.

#### Feature Engineering,  simple examples

In [0]:
## 예를 들어서 만들어 보자 I

## Timestamp_<lambda> 라는 컬럼에는 churn 기준일에 가까운 가장 마지막 거래일.
## Timestamp_nunique 는 방문한 날짜들의 수, row는 물건별로 기록되어 있다.
## Quantity_sum 에는 총 몇개를 구매했는지
## Quantity_max 는 한번에 최대 몇개씩 샀는지
## Quantity_mean 에는 평균 몇개씩 구매하는지
## ItemId_nunique에는, 몇 종류의 상품들을 구매했는지 담아본다.

fe1 = before_21.groupby(by = ['UserId'], as_index=False).agg({'Timestamp' : [lambda x : (churn_thres_day - x.max()).days, 'nunique'],
                                                       'Quantity' : ['sum', 'max', 'mean'],
                                                       'ItemId' : 'nunique'})

## 컬럼이름을 보기 좋게 바꿀 것이다. 주석을 달고 실행해보면 비교할 수 있다.
fe1.columns = ['_'.join(col).strip() for col in fe1.columns.values]

## 컬럼 이름을 한 번 더 바꾼다. 반복실행 에러 방지를 위해 if를 걸었다.
if 'UserId_' in fe1.columns :
    fe1.rename(columns={'UserId_':'UserId'}, inplace = True)


fe1.head()

#### Feature Engineering, difficult examples

그룹바이를 두번 이상 걸어야 하는 경우도 있을 수 있다.

In [0]:
## 코드 비교해보자.
# UserId 별, 구매일자별,  총 구매 수량과, 총 구매금액이다.
fe2_temp = before_21.groupby(by = ['UserId', 'Timestamp'], as_index = False).agg({'Quantity' : 'sum',
                                                                    'Value' : 'sum'})

fe2_temp1 = before_21.groupby(by = ['UserId', 'Timestamp'], as_index = False)[['Quantity', 'Value']].sum()

print(fe2_temp.head(3))
print(fe2_temp1.head(3))

In [0]:
# UserId 별, 구매일자별,  총 구매 수량과, 총 구매금액이다.
fe2_temp.head(3)

In [0]:
# UserId 별, 구매일자별,  총 구매 수량과, 총 구매금액이다.
if 'Quantity' in fe2_temp.columns :
    fe2_temp.rename(columns={'Quantity':'Quantity_sum_daily',
                       'Value' : 'Value_sum_daily'}, inplace = True)
fe2_temp.head(3)


In [0]:
## 방문일 간격에 대한 정보를 만들어 보자

# UserId별로, shift된 Timestamp를 만들어 붙이자.
fe2_temp['Time_lag'] = fe2_temp.groupby(by = ['UserId'],as_index=False)['Timestamp'].shift(periods=1)

# 재방문 간격을 계산한다.
fe2_temp['revisit_term'] = fe2_temp['Timestamp'] - fe2_temp['Time_lag']

# 재방문 간격을 int로 바꾼다. (원래는 timedelta type 이다.)
fe2_temp['revisit_term'] = fe2_temp['revisit_term'].dt.days

In [0]:
fe2_temp.head()

In [0]:
fe2 = fe2_temp.groupby(by=['UserId'], as_index=False).agg({'Quantity_sum_daily' : ['mean','std'],
                                                         'Value_sum_daily' : ['mean', 'std'],
                                                         'revisit_term' : 'mean'})

## 컬럼이름을 보기 좋게 바꿀 것이다. 주석을 달고 실행해보면 비교할 수 있다.
fe2.columns = ['_'.join(col).strip() for col in fe2.columns.values]

## 컬럼 이름을 한 번 더 바꾼다. 반복실행 에러 방지를 위해 if를 걸었다.
if 'UserId_' in fe2.columns :
    fe2.rename(columns={'UserId_':'UserId'}, inplace = True)

print('NaN 이 나오는 것 들에 대해서 고민해보자. 위의 Timestamp_nunique가 1일 때와 엮어서 해석해야 한다')
print('NaN이 나오는 경우, 어쩔 수 없이 0으로 대체를 해야만 하는데...')
print('지금 만들어낸 Feature들은, Timestamp_nunique와 상호작용이 일어나고 있다 ==> Feature간 상호작용 고려하지 않는 선형모델을 쓰면 망가질 수 있다.')
print('선형모델 순진하게 사용하면? ==> 0으로 대체된 NaN의 해석이 꼬여버린다.')

fe2.head()


In [0]:
fe2 = fe2.fillna(0)
fe2.head()

### Feature Engineering, your turn!

직접 해봐야한다!

여러분의 아이디어로!

## Data Preprocessing IV : Join

내 손에 쥐어진 데이터, 다시 한 번 확인해보자.

다음의 4개 (실습을 했다면 4개 초과) 테이블을 Join 해야 한다.

* user_info
* labels
* fe1
* fe2

In [0]:
user_info.head(2)

In [0]:
labels.head(2)

In [0]:
fe1.head(3)

In [0]:
fe2.head(3)

한꺼번에 Join걸 수도 있지만... 차근차근하자.

In [0]:
temp_join1 = pd.merge(labels, user_info, how = 'left')
print(temp_join1.shape)
temp_join1.head()

In [0]:
temp_join2 = pd.merge(temp_join1, fe1, how = 'left')
temp_join_fin = pd.merge(temp_join2, fe2, how = 'left')

print(temp_join_fin.shape)
temp_join_fin.head()

In [0]:
# Null check
temp_join_fin.isnull().sum()

## Data Preprocessing V : Dummy

In [0]:
# churn = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurnTemplate_FeatureEngg.csv')
'''
아래의 코드는 row를 셔플하기 위해 필요하다.
'''
churn = temp_join_fin

churn = churn.sample(frac = 1, random_state = 2018).reset_index(drop=True)

churn.head()

In [0]:
# User는 따로 떼어 둬야 어떤 User인지 알 수 있다.
Users = churn[['UserId']]

# Unknown 투성이인 Gender 와 UserType을 버리자
churn_dropped = churn.drop(['UserId'], axis=1)
churn_dropped.head()

더미를 이용하여 펼치자.

컴퓨터는 A, B, C, D라고 입력되어 있어봤자 그걸 그대로 인식할 수 없다!

In [0]:
pd.get_dummies(churn_dropped[['Age','Address']]).head()

In [0]:
churn_processed = pd.concat([churn_dropped, pd.get_dummies(churn_dropped[['Age','Address']])], axis=1).drop(['Age','Address'], axis=1)

churn_processed.head()

## Train set, Test set

시계열은 아니다!

데이터의 모양을 먼저 본다

In [0]:
churn_processed.shape

In [0]:
X_train, X_test, Y_train, Y_test = churn_processed.loc[:7000].drop('label', axis=1), \
                                    churn_processed.loc[7001:].drop('label', axis=1),\
                                    churn_processed.loc[:7000, 'label'],\
                                    churn_processed.loc[7001:, 'label']

In [0]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

## Logistic Regression

In [0]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}%'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}%'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
pd.DataFrame({'colnames':X_train.columns, 'coef':clf.coef_.reshape(-1,)})

## K- nearest Neighbor  Classification

In [0]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = KNeighborsClassifier(n_neighbors = 30)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))


## Random Forest

[공식 사이킷 런 문서](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) 를 참고하여 튜닝해보자.

반드시 알아야 할 것은 다음과 같다.

* n_estimators
* max_depth
* min_samples_split
* min_samples_leaf
* max_leaf_nodes


In [0]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = RandomForestClassifier(n_estimators = 30,
                            max_depth = None,
                            min_samples_split = 5,
                            min_samples_leaf = 1,
                            max_leaf_nodes = None)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
